### Suposiciones
- La red es fija y la tabla se llama red
- La tabla de vértices se llama red_vertices_pgr
- tiene una columna que se llama costo
- tiene campos source y target
- todas las columnas de geometría se llaman geom

Primero todos los imports

In [1]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from functools import reduce
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.transform import from_origin
import io
from subprocess import call


### Parámetros de la conexión
Aquí servidor, usuario, password y base de datos están fijos, en realidad hay que resolver cómo funciona esto en la aplicación

In [2]:
db = 'cadenas'
usr = 'postgres' 
psw = 'postgres' 
host = '192.168.18.22'

In [3]:
con = psycopg2.connect(database= db, user=usr, password=psw, host=host)
engine = create_engine('postgresql://{}:{}@{}:5432/{}'.format(usr,psw,host,db))

### Etapas de la cadena
Variable con los nombre de las tablas de etapas. Esta tiene que venir de la interfase

In [4]:
etapas = ['etapa1', 'etapa2', 'etapa3', 'etapa4' ]

### Relaciones entre nodos y etapas

Asignar a cada punto de cada etapa el nodo más cercano de la red. Se crean las tablas con las relaciones:`etapa_i_node`, cada tabla tiene dos columnas, el id de la tabla de etapas y el id de la tabla de nodos.


In [5]:
def node_relations(table, engine, connection):
    """Toma una tabla de puntos y le agrega una columna con el id del
       nodo más cercano de la red.
       Elimina la tabla de relaciones en caso de que exista.
       Regresa el dataframe con la relación.
    """
    drop_qry = """drop table if exists %(tabla)s_node""" % {"tabla": table}
    curs = connection.cursor()
    curs.execute(drop_qry)
    connection.commit()
    sql = """
            select f.id as id_%(tabla)s, (
              SELECT n.id
              FROM red_vertices_pgr As n
              ORDER BY f.geom <-> n.geom LIMIT 1
            )as closest_node
            from %(tabla)s f
          """ % {"tabla": table}
    try:
        df = pd.read_sql(sql, engine)
    except ValueError as e:
        print(e)
    try:
        df.to_sql(table + '_node', engine)
    except ValueError as e:
        print(e)
    return df

Entonces, para agregar todas las tablas de relaciones hacemos:

In [6]:
node_relations_list = []
for etapa in etapas:
    node_relations_list.append(node_relations(etapa, engine, con))

In [8]:
node_relations_list[-1].head()

,id_etapa4,closest_node
0,1,6156
1,2,1626
2,3,2827
3,4,5319
4,5,13025


### Costos por etapa

Primero la función que calcula el costo entre etapas sucesivas (esto asume que la lista de etapas está ordenada)

In [9]:
def stage_cost(source_table, target_table, cost_column):
    params = {'source': source_table, 'target': target_table, 'cost': cost_column }
    qry_str = """SELECT DISTINCT ON (start_vid)
                 start_vid, end_vid, agg_cost as costo_%(source)s_%(target)s
          FROM   (SELECT * FROM pgr_dijkstraCost(
              'select id, source, target, %(cost)s as cost from red',
              array(select distinct(s.closest_node) from (select e.*, r.closest_node
                                                        from %(source)s e
                                                        join %(source)s_node r
                                                        on e.id = r.id_%(source)s::int) as s),
              array(select distinct(t.closest_node) from (select e.*, r.closest_node
                                                        from %(target)s e
                                                        join %(target)s_node r
                                                        on e.id = r.id_%(target)s::int) as t),
                 directed:=false)
          ) as sub
          ORDER  BY start_vid, agg_cost asc""" % params
    try:
        df = pd.read_sql(qry_str, engine)
    except ValueError as e:
        print(e)
    return df

Para calcular la distancia entre dos etapas hacemos:

In [10]:
d0 = stage_cost("etapa1", "etapa2", "costo")
d0.head()

,start_vid,end_vid,costo_etapa1_etapa2
0,412,1851,268660.672500
1,459,1851,223833.071148
2,467,1851,233672.559784
3,638,11230,100905.641581
4,639,11230,105890.189821


Y para calcular todas las distancias:

In [11]:
distancias = []
for i, etapa in enumerate(etapas): 
    if i < len(etapas)-1:
        distancias.append(stage_cost(etapa,etapas[i+1], "costo"))

In [17]:
len(distancias[0])#.head()

1676

Ahora hay que unir cada resultado a la geometría que le corresponde, esto es:

- etapa1 -> costo de 1 a 2
- etapa2 -> costo de 2 a 3
- etapa(k-1) -> costo de k-1 a k

Primero leemos los datos originales

In [18]:
etapas_gdfs = []
for etapa in etapas:
    sql = """select * from %(etapa)s""" % {"etapa":etapa}
    etapas_gdfs.append(gpd.GeoDataFrame.from_postgis(sql, con, geom_col='geom'))
#etapas_gdfs[0].head()

,id_0,geom,id,id_ac
0,1,POINT (645736.3108251573 1808481.380637101),1737,07052028-2002
1,2,POINT (520212.1233636757 1896539.235540587),1,07118002-1011
2,3,POINT (551019.6914652064 1860672.211684213),2,07093002-0001
3,4,POINT (587381.6952086436 1652313.728707811),3,07102002-1010
4,5,POINT (552043.9602639768 1892579.33194623),4,07081003-4012


Luego, para todas las etapas, menos la última, unimos las geometrías con los costos. Pero primero a través de las relaciones entre nodos y etapas:

In [19]:
etapas_costo = []
for i, (etapa,relacion) in enumerate(zip(etapas, node_relations_list)):
    if i < len(node_relations_list) - 1:
        etapas_costo.append(pd.merge(relacion, distancias[i], 
                                     left_on='closest_node', 
                                     right_on='start_vid', how='right'))       

Creamos una lista de GeoDataFrames con los costos por etapa

In [20]:
gdfs_costo = []
for i, (etapa, costo) in enumerate(zip(etapas[0:-1], etapas_costo)):
    gdfs_costo.append(pd.merge(etapas_gdfs[i], costo, left_on='id', right_on='id_' + etapa, how='right'))

3

Ahora creamos un nuevo geoDataframe con valores de los costos por etapa y total

In [30]:
costos_totales = reduce(lambda x, y,: pd.merge(x, y, left_on='end_vid', 
                                               right_on='start_vid', 
                                               how='left',
                                               suffixes=['','_y']), gdfs_costo)
#costos_totales = reduce(lambda x, y: pd.merge(x, y, left_on='start_vid', right_on='end_vid', how='right'), gdfs_costo)
#costos_totales["cost_tot"] = costos_totales.iloc[:, costos_totales.columns.str.contains('costo_')].sum(1)
#pd.merge(gdfs_costo[0],gdfs_costo[1],left_on='end_vid', 
 #        right_on='start_vid', 
  #       how='left',
  #       suffixes=['','_y']).head()

2647

In [24]:
for i,etapa in enumerate(etapas[:-1]):
    costos_totales["cost_"+etapas[0] + "_" + etapas[i+1]] = \
    costos_totales.iloc[:, costos_totales.columns.str.contains('costo_')].sum(1)
    print("costo_" + etapa + "_" + etapas[i+1])

costo_etapa1_etapa2
costo_etapa2_etapa3
costo_etapa3_etapa4


In [25]:
nuevas_columnas = [c for c in costos_totales.columns if c.startswith("costo")]
nuevas_columnas = ["id_" + etapas[0], "geom"] + nuevas_columnas 
solo_costos = costos_totales[nuevas_columnas]
len(solo_costos)

2647

In [46]:
sumas_acumuladas = solo_costos.iloc[:, solo_costos.columns.str.contains('costo_')].cumsum(axis=1)
sumas_acumuladas = sumas_acumuladas.merge(solo_costos.iloc[:,[0,1]], left_index=True, right_index=True)
sumas_acumuladas = gpd.GeoDataFrame(sumas_acumuladas, geometry = sumas_acumuladas.geom)
sumas_acumuladas.to_file(driver= 'ESRI Shapefile', 
                         filename= "/home/eurekastein/Documentos/cadenas.py/idw/result.shp") 

/home/eurekastein/anaconda3/envs/cadenas/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
CPLE_NotSupported in Normalized/laundered field name: 'costo_etapa1_etapa2' to 'costo_etap'
CPLE_NotSupported in Normalized/laundered field name: 'costo_etapa2_etapa3' to 'costo_et_1'
CPLE_NotSupported in Normalized/laundered field name: 'costo_etapa3_etapa4' to 'costo_et_2'


ValueError: Invalid field type <class 'shapely.geometry.point.Point'>

In [45]:
for columna in sumas_acumuladas:
    if columna.startswith('costo_'):
            comando = ['gdal_grid', '-zfield', 'costo_tota', '-l', 'costo', '-a',
               'invdist:power=2.0:smothing=0.0:radius1=0.0:radius2=0.0:angle=0.0:max_points=0:min_points=0:nodata=0.0',
               '-of', 'GTiff', '/home/eurekastein/Documentos/cadenas.py/idw/costo'+ columna+'.shp', 
               '/home/eurekastein/Documentos/cadenas.py/idw/costo'+ columna+'.tif']
    try:
        call(comando)
        print(comando)
    except:
        print('valió verga')
    print('La columna '+columna+' no valió verga')
   

['gdal_grid', '-zfield', 'costo_tota', '-l', 'costo', '-a', 'invdist:power=2.0:smothing=0.0:radius1=0.0:radius2=0.0:angle=0.0:max_points=0:min_points=0:nodata=0.0', '-of', 'GTiff', '/home/eurekastein/Documentos/cadenas.py/idw/costocosto_etapa1_etapa2.shp', '/home/eurekastein/Documentos/cadenas.py/idw/costocosto_etapa1_etapa2.tif']
La columna costo_etapa1_etapa2 no valió verga
['gdal_grid', '-zfield', 'costo_tota', '-l', 'costo', '-a', 'invdist:power=2.0:smothing=0.0:radius1=0.0:radius2=0.0:angle=0.0:max_points=0:min_points=0:nodata=0.0', '-of', 'GTiff', '/home/eurekastein/Documentos/cadenas.py/idw/costocosto_etapa2_etapa3.shp', '/home/eurekastein/Documentos/cadenas.py/idw/costocosto_etapa2_etapa3.tif']
La columna costo_etapa2_etapa3 no valió verga
['gdal_grid', '-zfield', 'costo_tota', '-l', 'costo', '-a', 'invdist:power=2.0:smothing=0.0:radius1=0.0:radius2=0.0:angle=0.0:max_points=0:min_points=0:nodata=0.0', '-of', 'GTiff', '/home/eurekastein/Documentos/cadenas.py/idw/costocosto_etapa

Nuevo intento de IDW

In [31]:
def readPoints(dataFile, Zfield='Z'):
    data = {}
    xv=[]
    yv=[]
    values=[]
    ds = ogr.Open(dataFile)
    if ds is None:
       raise Exception('Could not open ' + dataFile)
    
    layer = ds.GetLayer()
    proj = layer.GetSpatialRef()
    extent = layer.GetExtent()

    feature = layer.GetNextFeature()
    if feature.GetFieldIndex(zField) == -1:
         raise Exception('zField is not valid: ' + zField)

    while feature:
        geometry = feature.GetGeometryRef()
        xv.append(geometry.GetX())
        yv.append(geometry.GetY())
        values.append(feature.GetField(zField))

        feature = layer.GetNextFeature()
    data['extent'] = extent 
    data['xv']=xv
    data['yv']=yv
    data['values']=values
    data['proj'] = proj
    ds = None
    return data